<a href="https://colab.research.google.com/github/Prakrut/Fraud_Detection/blob/master/Code/CreditCardFraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries 

In [1]:
import sys, os
print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['C:\\Users\\prakr\\PycharmProjects\\Fraud_Detection', 'C:/Users/prakr/PycharmProjects/Fraud_Detection'])


Python 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)] on win32


In [4]:
for i in os.listdir('..\\'):
    print(i)

.git
.gitignore
.idea
Code
Data
README.md
requirements.txt
venv


In [13]:
path = '..\\venev\\'

In [3]:
# Python
import pandas as pd

# Thir-Party
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Custom